In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import PeftModel, PeftConfig
import torch
import gradio as gr

d_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
local_model_path = "outputs/checkpoint-20"     # Path to the combined weights

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [2]:
# Loading the base Model
config = PeftConfig.from_pretrained(local_model_path)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True,
    torch_dtype=torch.float16,
    # quantization_config=bnb_config,
    device_map=d_map,
    # trust_remote_code=True
)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
# # load the base model with the Lora model
# model = PeftModel.from_pretrained(model, local_model_path)
# mergedModel = model.merge_and_unload()
# # mergedModel.eval()

# model.eval()

for name, param in model.named_parameters():
    print(f"Layer: {name}, Parameter Shape: {param.shape}")

Layer: model.embed_tokens.weight, Parameter Shape: torch.Size([32000, 4096])
Layer: model.layers.0.self_attn.q_proj.weight, Parameter Shape: torch.Size([4096, 4096])
Layer: model.layers.0.self_attn.k_proj.weight, Parameter Shape: torch.Size([1024, 4096])
Layer: model.layers.0.self_attn.v_proj.weight, Parameter Shape: torch.Size([1024, 4096])
Layer: model.layers.0.self_attn.o_proj.weight, Parameter Shape: torch.Size([4096, 4096])
Layer: model.layers.0.mlp.gate_proj.weight, Parameter Shape: torch.Size([14336, 4096])
Layer: model.layers.0.mlp.up_proj.weight, Parameter Shape: torch.Size([14336, 4096])
Layer: model.layers.0.mlp.down_proj.weight, Parameter Shape: torch.Size([4096, 14336])
Layer: model.layers.0.input_layernorm.weight, Parameter Shape: torch.Size([4096])
Layer: model.layers.0.post_attention_layernorm.weight, Parameter Shape: torch.Size([4096])
Layer: model.layers.1.self_attn.q_proj.weight, Parameter Shape: torch.Size([4096, 4096])
Layer: model.layers.1.self_attn.k_proj.weight,

In [20]:
def inferance(query: str, model, tokenizer, temp = 1.0, limit = 200) -> str:
  device = "cuda:0"

  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  {query}

  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=int(limit), temperature=temp, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [21]:
def predict(temp, limit, text):
    prompt = text
    out = inferance(prompt, mergedModel, tokenizer, temp = 1.0, limit = 200)
    return out

pred = gr.Interface(
    predict,
    inputs=[
        gr.Slider(0.001, 10, value=0.1, label="Temperature"),
        gr.Slider(1, 1024, value=128, label="Token Limit"),
        gr.Textbox(
            label="Input",
            lines=1,
            value="#### Human: What's the capital of Australia?#### Assistant: ",
        ),
    ],
    outputs='text',
)

pred.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://7443cd2e64f1b47d4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
from datasets import load_dataset

In [12]:
data = load_dataset("gbharti/finance-alpaca", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
data[1021]

{'output': "This is definitely a scam. I had a friend sign up for a very similar offer and what they did was send a fake check and then asked to transfer the same amount to them. So now you just send them a couple grand and you're holding a fake check.",
 'text': '',
 'instruction': 'What risks are there acting as a broker between PayPal and electronic bank transfers?',
 'input': ''}